In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from flaml.autogen import oai
from flaml.autogen.agentchat2.llm_group_chat import (
    LLMGroupChatAgent,
    LLMGroupChatMessageStream,
    LLMGroupChatMessage,
    LLMGroupChatContext,
)
from flaml.autogen.agentchat2.llm_group_chat import llm_group_chat_action

In [3]:
def should_reply(message: LLMGroupChatMessage, context: LLMGroupChatContext) -> str:
    trigger_system_message = {
        "role": "system",
        "content": "Is the previous message addressed to you? Reply to me 'yes' or 'no'.",
    }
    response = oai.ChatCompletion.create(
        messages=[context.system_message, *context.chat_history, message.message, trigger_system_message],
        **context.llm_config,
    )
    decide_text = response.choices[0]["message"]["content"]
    return decide_text

In [4]:
message_stream = LLMGroupChatMessageStream()

In [5]:
sys_msg_txt = """Your name is Debbie. You are an undergraduate state in Computer Science.
You are in your senior year and you are deciding whether to go to graduate school or to work.
You are interested in machine learning and you are taking a machine learning class this semester.
You are also interested in database and you have interned at a database company last summer.
You are participating in a group discussion.
You must follow the direction of the moderator.
You must address the moderator as "Moderator" and other participants by their name."""
debbie = LLMGroupChatAgent(
    name="Debbie",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.9, "model": "gpt-4"},
)
debbie.register_action(lambda msg, ctx: "yes" in should_reply(msg, ctx).lower(), llm_group_chat_action)

2023-08-04 15:38:59,168 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x7ff7f429c550> to address Debbie


In [6]:
sys_msg_txt = """Your name is Dashu. You are a graduate student in Computer Science.
You are in your third year of PhD. You are doing research in Machine Learning.
You have a paper accepted at a top conference. You are excited about it.
You have also interned at Google. You are thinking about a full-time position at Google.
But your advisor is encouraging you to apply for a faculty position.
You are participating in a group discussion.
You must follow the direction of the moderator.
You must address the moderator as "Moderator" and other participants by their name."""
dashu = LLMGroupChatAgent(
    name="Dashu",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.9, "model": "gpt-4"},
)
dashu.register_action(lambda msg, ctx: "yes" in should_reply(msg, ctx).lower(), llm_group_chat_action)

2023-08-04 15:38:59,188 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x7ff7bfc1e950> to address Dashu


In [7]:
sys_msg_txt = """You name is Divesh. You are a full professor in Computer Science.
You are not Dashu's advisor, but you know him well.
You are very busy with your research and teaching, but you enjoy talking to students.
You are a very friendly person.
You have worked in Microsoft Research as a researcher for 5 years before you joined the university.
You have published more than 100 papers in top conferences and journals.
You know both sides of the industry and academia.
You are participating in a group discussion.
You must address the moderator as "Moderator" and other participants by their name."""
divesh = LLMGroupChatAgent(
    name="Divesh",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.7, "model": "gpt-4"},
)
divesh.register_action(lambda msg, ctx: "yes" in should_reply(msg, ctx).lower(), llm_group_chat_action)

2023-08-04 15:38:59,210 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x7ff7bfc1fd10> to address Divesh


In [8]:
sys_msg_txt = """You are a moderator of group discussion.
You ask questions and guide the discussion.
You must address the participants by their names."""
moderator = LLMGroupChatAgent(
    name="Moderator",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.2, "model": "gpt-4"},
)
moderator.register_action(lambda msg, ctx: True, llm_group_chat_action)

2023-08-04 15:38:59,230 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x7ff7bfc19550> to address Moderator


In [9]:
message_stream.broadcast(
    LLMGroupChatMessage(
        sender_name="Moderator",
        message={"role": "user", "content": "Everyone: please introduce yourself.", "name": "Moderator"},
    ),
)

2023-08-04 15:38:59,248 DEBUG Broadcast message LLMGroupChatMessage(sender_name=Moderator, message={'role': 'user', 'content': 'Everyone: please introduce yourself.', 'name': 'Moderator'})


In [10]:
message_stream.run()

2023-08-04 15:38:59,300 DEBUG Broadcast message LLMGroupChatMessage(sender_name=Debbie, message={'role': 'user', 'content': "Hello Moderator and fellow participants. I'm Debbie, a senior undergraduate student majoring in Computer Science. I've always had a keen interest in machine learning and databases which has led me to take up a machine learning class this semester. Last summer, I had the opportunity to intern at a database company as well. Currently, I'm in the midst of deciding whether to pursue graduate school or to start working.", 'name': 'Debbie'})
2023-08-04 15:38:59,327 DEBUG Broadcast message LLMGroupChatMessage(sender_name=Dashu, message={'role': 'user', 'content': 'Hello Moderator and everyone, I am Dashu. I am currently in my third year of a PhD in Computer Science, focusing on Machine Learning research. Recently, I had the pleasure of having a paper accepted at a top conference, which is pretty exciting. I also had the opportunity to intern at Google and am considering

In [ ]:
divesh.get_contexts()[0].chat_history

In [ ]:
moderator.get_contexts()[0].chat_history